In [1]:
import numpy as np
np.set_printoptions(suppress=True)
from sklearn.decomposition import PCA, TruncatedSVD

# 理论方法

In [2]:
a = np.random.randint(0,10, size=(5,3))
a

array([[0, 9, 0],
       [6, 0, 9],
       [3, 2, 8],
       [8, 9, 5],
       [7, 4, 3]])

中心化处理

In [3]:
a_new = a - np.mean(a, axis=0)
a_new

array([[-4.8,  4.2, -5. ],
       [ 1.2, -4.8,  4. ],
       [-1.8, -2.8,  3. ],
       [ 3.2,  4.2,  0. ],
       [ 2.2, -0.8, -2. ]])

In [4]:
np.allclose(a_new.mean(axis=0), np.zeros(a_new.shape[1]))

True

协方差矩阵

In [5]:
cov = np.dot(a_new.T, a_new) / (a_new.shape[0] -1)
cov

array([[ 10.7 ,  -2.3 ,   4.75],
       [ -2.3 ,  16.7 , -11.75],
       [  4.75, -11.75,  13.5 ]])

协方差矩阵的特征值分解

In [6]:
eig_vals, eig_vecs = np.linalg.eig(cov)
eig_vals

array([28.31080211,  9.94935833,  2.63983956])

In [7]:
eig_vecs

array([[-0.26819041,  0.92200322, -0.2792561 ],
       [ 0.71098102,  0.38503359,  0.58843446],
       [-0.65006145,  0.04073331,  0.75878911]])

取前`n`个最大特征向量进行映射降维得到`PCA`结果

In [8]:
a_pca = np.dot(a_new, eig_vecs)
a_pca

array([[ 7.52374151, -3.01214092,  0.01790851],
       [-6.3347832 , -0.57882413, -0.12443632],
       [-3.45818848, -2.61549991,  1.13141181],
       [ 2.12791099,  4.56755137,  1.57780522],
       [ 0.14131918,  1.63891359, -2.60268921]])

核心思想：
- 协方差矩阵的特征值大小即原始矩阵在该方向上投影的方差，要使得投影的方差最大
- 也即求协方差矩阵的最大特征值
- 根据需求进行前几大特征值的选取，对应特征向量即为待投影方向
- 原始向量 @ 特征向量即为PCA的结果

# 通常不用以上步骤求而是用求解奇异值的方式
[原因](https://zhuanlan.zhihu.com/p/85701151)
- 这种纯数学的解法，在实际工程实践中有以下问题：在数据量很大时，把原始矩阵进行转置求协方差矩阵，然后再进行特征值分解是一个非常慢的过程。 
- 稳定性问题。可以看到X转置乘以X，如果矩阵有非常小的数，很容易在平方中丢失。
- 奇异值分解不要求原始矩阵为方阵，符合生活中的实际数据。

In [9]:
U, S, V = np.linalg.svd(a_new)
U

array([[-0.70701363,  0.47747183, -0.00551113,  0.51118317,  0.10398931],
       [ 0.59528601,  0.09175275,  0.03829379,  0.77563452, -0.18477196],
       [ 0.32496948,  0.41459798, -0.3481785 , -0.09802412,  0.76919852],
       [-0.19996196, -0.72402891, -0.48555075,  0.33395923,  0.29750471],
       [-0.01327991, -0.25979365,  0.80094658,  0.12627493,  0.52428058]])

In [10]:
S

array([10.64157923,  6.30852069,  3.24951662])

In [11]:
V

array([[ 0.26819041, -0.71098102,  0.65006145],
       [-0.92200322, -0.38503359, -0.04073331],
       [ 0.2792561 , -0.58843446, -0.75878911]])

In [12]:
# 这个值和上面的eig_vals是一样的，只是元素顺序可能有差别
np.square(S) / (a_new.shape[0] -1)

array([28.31080211,  9.94935833,  2.63983956])

In [13]:
# 这个结果应该跟a_pca的绝对值一样，不过元素排列上会有差异
a_pca_svd = np.dot(a_new, V.T)
a_pca_svd

array([[-7.52374151,  3.01214092, -0.01790851],
       [ 6.3347832 ,  0.57882413,  0.12443632],
       [ 3.45818848,  2.61549991, -1.13141181],
       [-2.12791099, -4.56755137, -1.57780522],
       [-0.14131918, -1.63891359,  2.60268921]])

# `sklearn`中的`pca`也是通过`svd`实现的

In [16]:
pca = PCA(2)
# pca自动处理中心化
pca.fit_transform(a_new)

array([[ 7.52374151, -3.01214092],
       [-6.3347832 , -0.57882413],
       [-3.45818848, -2.61549991],
       [ 2.12791099,  4.56755137],
       [ 0.14131918,  1.63891359]])

截断svd的值取的是右奇异矩阵`V`的前`n`列

In [15]:
trc = TruncatedSVD(2)
trc.fit(a_new)
u = trc.components_
u

array([[-0.26819041,  0.71098102, -0.65006145],
       [ 0.92200322,  0.38503359,  0.04073331]])

In [17]:
# 结果和PCA应该相同
a_new @ u.T

array([[ 7.52374151, -3.01214092],
       [-6.3347832 , -0.57882413],
       [-3.45818848, -2.61549991],
       [ 2.12791099,  4.56755137],
       [ 0.14131918,  1.63891359]])